In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.preprocessing import StandardScaler
import folium
from folium.plugins import HeatMap
import seaborn as sns


In [2]:
import leafmap.foliumap as leafmap

# Load your dataset
df = pd.read_csv('tows.csv')

# Drop rows with missing 'Trade Name', 'lat', or 'lon'
df_clean = df.dropna(subset=['Trade Name', 'lat', 'lon'])

# Identify the top 5 towing companies by count
top_companies = df_clean['Trade Name'].value_counts().head(10).index

# Filter the dataset for the top 5 companies
df_top10 = df_clean[df_clean['Trade Name'].isin(top_companies)]

# Create a base map centered around the area of interest
map_center = [39.05, -77.10]  # Example coordinates for Montgomery County, MD
m = leafmap.Map(location=map_center, zoom_start=11)

# Define a color palette for the top 5 companies
colors = sns.color_palette("hsv", len(top_companies)).as_hex()

# Add a heatmap layer for each company
for company, color in zip(top_companies, colors):
    company_data = df_top10[df_top10['Trade Name'] == company]
    heat_data = company_data[['lat', 'lon']].values.tolist()
    HeatMap(heat_data, radius=10, gradient={0.4: color}).add_to(folium.FeatureGroup(name=company).add_to(m))

# Add layer control to toggle heatmaps
folium.LayerControl().add_to(m)

# Create a legend dictionary
for company, color in zip(top_companies, colors):
    legend_dict = {company: color for company, color in zip(top_companies, colors)}

# Add the legend to the map
m.add_legend(title='Towing Companies', legend_dict=legend_dict)

# Save the map to an HTML file
m.save('top10_towing_companies_heatmap.html')
